# O dataset


O dataset usado nessa aula foi inspirado neste [tutorial](https://www.kaggle.com/c/word2vec-nlp-tutorial#part-1-for-beginners-bag-of-words).
   
Conforme vamos ver em breve, o dataset é constituido por aproximadamente 50000 dados de reviews do IMDB, sendo 25000 dados para treino. Estes já foram pré processados e separados em reviews positivos (nota >=7) e negativos (<5).
    
Os outros 25000 não estão caracterizados, ilustrando o nosso desafio ! Uma vez completado com um modelo bacana, temos mais um dataset, de 50000 dados que será utilizado para a submissão no Kaggle.
    
## O conjunto de dados

Os dados estão estruturados da seguinte forma:
- id: um identificador do review
- sentimento: a classificação do review, sendo 0 negativo ou 1 positivo
- review: o review em si
        
## Uma análise exploratória dos dados


In [3]:
import pandas as pd 
#quoting = 3 fala pro python ignorar double quotes, pra gente n ter problema pra ler esse arquivo
train = pd.read_csv("..\data\labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [4]:
train.head()

id  sentiment                                             review
0  "5814_8"          1  "With all this stuff going down at the moment ...
1  "2381_9"          1  "\"The Classic War of the Worlds\" by Timothy ...
2  "7759_3"          0  "The film starts with a manager (Nicholas Bell...
3  "3630_4"          0  "It must be assumed that those who praised thi...
4  "9495_8"          1  "Superbly trashy and wondrously unpretentious ...

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [6]:
train['review'][10]

'"What happens when an army of wetbacks, towelheads, and Godless Eastern European commies gather their forces south of the border? Gary Busey kicks their butts, of course. Another laughable example of Reagan-era cultural fallout, Bulletproof wastes a decent supporting cast headed by L Q Jones and Thalmus Rasulala."'

In [4]:
train.shape

(25000, 3)

# Pré processamento é necessário

- Percebemos com esse review que talvez existam algumas coisas que seria legal removermos, tais como tags HTML ou até algumas outras palavras.

In [12]:
#Remover tags html
from bs4 import BeautifulSoup

example = BeautifulSoup(train["review"][0], "html5lib" ) 

In [13]:
example.get_text() 

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

Podemos ver que ao chamar o método get_text() do beautifulsoup ele remove as tags HTML pra gente

É uma biblioteca extremamente poderosa quando lidamos com dados vindos da web

# Removendo pontuações, stop words etc

- Um pouco de revisão

![Regexp Data Camp](img/regexp.png)

In [14]:
import re


re.sub('\w+',' ','rei leao 29@')

'     @'

In [15]:
import re

Uma fonte bem legal para validar suas regex é o [Pythex](https://pythex.org/)

In [29]:
##PEGUE APENAS LETRAS
string = example.get_text() 
letters_only = re.sub("\W+", ' ', string)

In [30]:
lower_case = letters_only.lower()
lower_case

' with all this stuff going down at the moment with mj i ve started listening to his music watching the odd documentary here and there watched the wiz and watched moonwalker again maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography part feature film which i remember going to see at the cinema when it was originally released some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for 20 minutes 

In [31]:
rmv_whitespace = lower_case.strip()
rmv_whitespace

'with all this stuff going down at the moment with mj i ve started listening to his music watching the odd documentary here and there watched the wiz and watched moonwalker again maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography part feature film which i remember going to see at the cinema when it was originally released some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for 20 minutes o

In [32]:
words = rmv_whitespace.split()
words[:5]

['with', 'all', 'this', 'stuff', 'going']

# Contando as palavras

In [33]:
from collections import Counter
#TO DO
counter = Counter(words)
print(counter)

Counter({'the': 19, 'is': 16, 'this': 11, 'mj': 11, 'of': 11, 'i': 10, 'and': 10, 'a': 10, 'to': 9, 'he': 9, 'it': 8, 'for': 5, 'people': 5, 'with': 4, 'all': 4, 'in': 4, 'or': 4, 's': 4, 'but': 4, 'so': 4, 'most': 4, 'one': 4, 'going': 3, 'his': 3, 'maybe': 3, 'just': 3, 'who': 3, 'which': 3, 'when': 3, 'bad': 3, 'like': 3, 'movie': 3, 'sequence': 3, 'know': 3, 'at': 2, 've': 2, 'music': 2, 'watched': 2, 'moonwalker': 2, 'into': 2, 'guy': 2, 'was': 2, 'really': 2, 'cool': 2, 'guilty': 2, 'part': 2, 'feature': 2, 'film': 2, 'some': 2, 'about': 2, 'also': 2, 'message': 2, 'drugs': 2, 'are': 2, 'michael': 2, 'jackson': 2, 'you': 2, 'then': 2, 'hate': 2, 'an': 2, 'fans': 2, 'that': 2, 'if': 2, 'nice': 2, 'on': 2, 'joe': 2, 'pesci': 2, 'as': 2, 'because': 2, 'whole': 2, 'not': 2, 'well': 2, 'stuff': 1, 'down': 1, 'moment': 1, 'started': 1, 'listening': 1, 'watching': 1, 'odd': 1, 'documentary': 1, 'here': 1, 'there': 1, 'wiz': 1, 'again': 1, 'want': 1, 'get': 1, 'certain': 1, 'insight': 1,

In [34]:
counter.most_common(10)

[('the', 19),
 ('is', 16),
 ('this', 11),
 ('mj', 11),
 ('of', 11),
 ('i', 10),
 ('and', 10),
 ('a', 10),
 ('to', 9),
 ('he', 9)]

Como podemos ver as palavras mais comuns são palavras que não dizem muito para a gente sobre o significado geral das reviews: **the**; **this**; **a**; **is**. Em NLP e áreas relacionadas, essas palavras são chamadas de _stop words_. Podemos remover elas usando uma biblioteca famosa para lidar com dados de NLP: o NLTK

In [35]:
import nltk
#nltk.download() para baixar as stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tati\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
from nltk.corpus import stopwords # Import the stop word list
stopwords.words("english") 

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [30]:
stopwords.words("portuguese")

['de',
 'a',
 'o',
 'que',
 'e',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'está',
 'estamos',
 'estão',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estávamos',
 'estavam',
 'estivera',
 'es

Se analisarmos as stopwords, dependendo do tipo de atividade que a gente quer fazer, podemos ver a presença de algumas stopwords como: **don**(de don't); **nor**; **no**; **do** e **does**. Considerando que estamos trabalhando como _Analise de Sentimentos_, esse tipo de palavra chave deveria ser removida?


Para o trabalho atual o contexto das palavras acaba não afetando muito, mas para uma aplicação mais real isso pode fazer diferença. Como contornar esse problema?

## Mas e no português?

No português, apesar de termos os processos de pré processamento bem semelhantes, é bom lembrar que também temos casos de caracteres espeicias como acentos ou ç. Para contornar isso, basta usar o normalize da biblioteca unicode data.

In [31]:
import unicodedata

text = "coração"

#TODO

'coracao'

In [37]:
#TODO
no_stops = [w for w in words if w not in stopwords.words('english')]

Counter(no_stops).most_common(10)

[('mj', 11),
 ('people', 5),
 ('one', 4),
 ('going', 3),
 ('maybe', 3),
 ('bad', 3),
 ('like', 3),
 ('movie', 3),
 ('sequence', 3),
 ('know', 3)]

Dado que os nossos dados tratam de filmes, esses conjuntos fazem um pouco mais de sentido, certo? Mas temos muito mais trabalho pela frente !

Se dermos uma olhada nas palavras que compõe words, vamos ver casos como **watching** e **watched**, entre outras. Como queremos capturar o significado da palavra, podemos reduzir elas ao seu estado raíz, que é **watch**; Ou para palavras como **good**, **better** e **well**, seria legal agrupá-las para o radical **good**.

Esse processo é conhecido por dois métodos: o _Stemming_ e o _Lemmatisation_

### Stemming

- É o processo de agrupar palavras, reduzindo elas a sua forma raíz. É um processo mais algoritmico. Por exemplo: se a palavra termina em **ed**, remove o **ed** pq a chance de ser um verbo é alta. No caso, palavras como _watched_ seriam reduzidas para _watch_

- O mais famoso algoritmo de Stemming considerando o inglês é o de Porter.


### Lemmatisation

- É o processo de agrupar palavras de forma que elas possam ser analizadas como um termo único, um lemma, ou sua forma no dicionário. Diferente do Stemming, ela leva em conta o contexto, sendo necessário identificar o _Part of Speech_ (categorizar a palavra gramaticalmente), e o contexto, de forma a ser pego o real da palavra da sequência.
    - **good**; **better**; **well** são reduzidos para **good** é um exemplo de Lemma.
    
Como é possível ver, o Lemmatisation é mais difícil de realizar do que o Stemming, sendo uma área bem aberta de pesquisa já que ainda não existe uma forma ideal, canônica de fazer isso.

O grande problema do Stemming é que ele pode reduzir palavras de forma que elas percam um pouco do significado ao removermos os radicais. Um exemplo é a palavra **bobalhões** que ao ser apresentada pelo radical fica como **bobalhõ**, enquanto seu lemma seria **bobalhão** ou, dependendo, até **bobo**.

Como é possível ver, por considerar as organizações morfológicas de cada língua, não existe nenhum lemma ou stemming universal.

Considerando o português, por ser uma língua bem mais complexa, ainda não existe uma referência boa como o caso do WordNetLemmatizer, que é baseado no [WordNet](https://wordnet.princeton.edu/), um banco de dados léxico de Princeton. Para os casos de Stemming, o mais famoso (inclusive é o que usamos aqui no Elo7) é o Stemmer da Vivane Orengo, da UFRGS, o [RSLP Stemmer](http://www.inf.ufrgs.br/~viviane/rslp/index.htm), implementado no NLTK.

Outro projeto bem legal para Stemmer é o [Snowball](http://snowballstem.org/) que apresenta diversos algoritmos de diversas linguas para o Stemming.

In [42]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tati\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
from nltk.stem import WordNetLemmatizer #WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [44]:
stemming = SnowballStemmer("english")

stemmed = [stemming.stem(w) for w in no_stops]

# Create the bag-of-words: bow
bow = Counter(stemmed)

# Print the 10 most common tokens
print(bow.most_common(10))

[('mj', 11), ('peopl', 5), ('one', 4), ('go', 3), ('watch', 3), ('mayb', 3), ('want', 3), ('film', 3), ('messag', 3), ('drug', 3)]


Como é possível ver para **esse caso**, não fez muita diferença a mudança entre o uso de um Stemming e um Lemma. Para fins de compreensão das palavras, vamos, então utilizar o Lemmatisation.

Agora, é legal a gente ter uma função que faça toda a limpeza que a gente queira para o nosso texto.

In [51]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review, "html5lib").get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("\W+", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().strip().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Apply Lemma
    lemma = [stemming.stem(t) for t in meaningful_words]
    # 7. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( lemma ))   

In [52]:
review_to_words( train["review"][0] )

'stuff go moment mj start listen music watch odd documentari watch wiz watch moonwalk mayb want get certain insight guy thought realli cool eighti mayb make mind whether guilti innoc moonwalk part biographi part featur film rememb go see cinema origin releas subtl messag mj feel toward press also obvious messag drug bad kay visual impress cours michael jackson unless remot like mj anyway go hate find bore may call mj egotist consent make movi mj fan would say made fan true realli nice actual featur film bit final start 20 minut exclud smooth crimin sequenc joe pesci convinc psychopath power drug lord want mj dead bad beyond mj overheard plan nah joe pesci charact rant want peopl know suppli drug etc dunno mayb hate mj music lot cool thing like mj turn car robot whole speed demon sequenc also director must patienc saint came film kiddi bad sequenc usual director hate work one kid let alon whole bunch perform complex danc scene bottom line movi peopl like mj one level anoth think peopl s

### Aplicando o Pré Processamento em Todo o Dataset

In [53]:
#TODO
from tqdm import tqdm
num_reviews = train["review"].size

clean_train_review = []

for i in tqdm(range(num_reviews), total=num_reviews):
    clean_train_review.append(review_to_words (train["review"][i]))

100%|███████████████████████████████████| 25000/25000 [02:39<00:00, 156.57it/s]


In [56]:
clean_train_review[0:2]

['stuff go moment mj start listen music watch odd documentari watch wiz watch moonwalk mayb want get certain insight guy thought realli cool eighti mayb make mind whether guilti innoc moonwalk part biographi part featur film rememb go see cinema origin releas subtl messag mj feel toward press also obvious messag drug bad kay visual impress cours michael jackson unless remot like mj anyway go hate find bore may call mj egotist consent make movi mj fan would say made fan true realli nice actual featur film bit final start 20 minut exclud smooth crimin sequenc joe pesci convinc psychopath power drug lord want mj dead bad beyond mj overheard plan nah joe pesci charact rant want peopl know suppli drug etc dunno mayb hate mj music lot cool thing like mj turn car robot whole speed demon sequenc also director must patienc saint came film kiddi bad sequenc usual director hate work one kid let alon whole bunch perform complex danc scene bottom line movi peopl like mj one level anoth think peopl 

### Mas e agora? O computador lê o que a gente lê?

Não ! Como sabemos, os computadores conseguem interpretar apenas números. Mas como a gente faz isso? Um dos modelos mais famosos é o _Bag of Words_ (BoW). O modelo de _Bag of Words_ aprende o vocabulário a partir de todos os documentos e então, dada todas as palavras, considera cada documento como a presença ou ausência de cada palavra. Por exemplo, considere as duas sentenças:

In [101]:
# Sentenca 1
sentence_1 = "o rato roeu a roupa do rei de roma"
#Sentenca 2
sentence_2 = "o rato comeu o queijo e o sapato do joão"
sentences = [sentence_1,sentence_2]

A partir das duas sentenças, o vocabulário é _{o,rato,roeu,a,roupa,queijo, e, rei, de, do, roma, joão}_ . Assim, parar criarmos o nosso modelo de _BoW_, contamos o número de vezes que cada palavra aparece. Sendo assim, para cada uma das sentenças:
- vocabulário = {o,rato,roeu,a,roupa,queijo, e, rei, de, do, roma, joão, comeu}
- sentence_1 = { 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0 }
- sentence_2 = { 3, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1 }

Esses são os _features vectors_ de cada uma das sentenças. Esse conjunto de _feature vectors_ é o conhecido como _corpus_ (ou _term document matrix_)
Vamos ver como podemos realizar isso utilizando duas bibliotecas bem comum do _Python_ para _NLP_, o _gensim_ e o _scikit-learn_.

In [75]:
from gensim.corpora.dictionary import Dictionary

docs = [doc.split() for doc in clean_train_reviews]

In [76]:
docs[0][0:5]

['stuff', 'going', 'moment', 'mj', 've']

In [77]:
docs[1][0:5]

['classic', 'war', 'world', 'timothy', 'hines']

In [78]:
dictionary = Dictionary(docs)

In [79]:
dictionary.token2id.get("stuff"),dictionary.token2id.get("timothy"),dictionary.token2id.get("mj")

(0, 167, 3)

In [80]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [88]:
#eh o id da palavra e a frequencia dela no documento
corpus[0][:10]

[(0, 1),
 (1, 3),
 (2, 1),
 (3, 11),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 2),
 (8, 1),
 (9, 1)]

In [82]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)  
#Max Features considera só as 5000 top palavras, sendo mais um processo de pre processamento

train_data = vectorizer.fit(clean_train_reviews)

In [83]:
train_data.vocabulary_

{'stuff': 4295,
 'going': 1960,
 'moment': 2907,
 've': 4750,
 'started': 4221,
 'listening': 2640,
 'music': 2959,
 'watching': 4846,
 'odd': 3097,
 'documentary': 1345,
 'watched': 4844,
 'maybe': 2798,
 'want': 4823,
 'get': 1930,
 'certain': 751,
 'insight': 2322,
 'guy': 2037,
 'thought': 4496,
 'really': 3586,
 'cool': 1028,
 'eighty': 1467,
 'make': 2734,
 'mind': 2868,
 'whether': 4885,
 'guilty': 2032,
 'innocent': 2318,
 'part': 3205,
 'biography': 503,
 'feature': 1709,
 'film': 1745,
 'remember': 3655,
 'see': 3904,
 'cinema': 838,
 'originally': 3139,
 'released': 3640,
 'subtle': 4313,
 'message': 2838,
 'feeling': 1715,
 'towards': 4569,
 'press': 3407,
 'also': 216,
 'obvious': 3088,
 'drug': 1400,
 'bad': 400,
 'kay': 2471,
 'visually': 4791,
 'impressive': 2271,
 'course': 1056,
 'michael': 2847,
 'jackson': 2397,
 'unless': 4700,
 'remotely': 3662,
 'like': 2620,
 'anyway': 278,
 'hate': 2086,
 'find': 1753,
 'boring': 559,
 'may': 2797,
 'call': 668,
 'making': 2737

In [84]:
len(train_data.vocabulary_)

5000

In [85]:
# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Essa forma de estrutura de dado é a mais convencional de um _BoW_. Nela, cada linha é um documento e cada coluna, uma palavra. Como podemos ver, a coluna 4295 corresponde a palavra 'stuff' e sua presença no documento 0 é indicada ao acessarmos a posição 4295 da coluna 0. 

Fica claro aqui que o modelo _Bag of Words_ não leva em consideração a **posição** das palavras.

In [61]:
train_data_features[0][1960] # Vai do indice 4292 ate o 4295

3

In [122]:
train_data_features.shape

(25000, 5000)

25 mil documentos e 5000 feature, sendo que cada feature corresponde a uma palavra e a posição da matrix é conhecida como frequência **daquela** palavra **naquele** documento. Esse tipo de matriz também é conhecida como matriz de frequência (_term frequency matrix_), sendo denotada por $t_f$.

Vamos ordenar a frequência das palavras de um documento?

In [86]:
from collections import defaultdict
import itertools

doc = corpus[2]

# Sort the doc for frequency: bow_doc
bow_doc = sorted(doc, key=lambda w: w[1], reverse=True)

# Print the top 5 words of the document alongside the count
for word_id, word_count in bow_doc[:10]:
    print(dictionary.get(word_id), word_count)

animal 4
film 3
sabretooth 3
sabretooths 3
one 2
scene 2
primal 2
park 2
nature 2
fearsome 2


E se considerarmos **todos** os documentos?

In [87]:
# Create the defaultdict: total_word_count
total_word_count = defaultdict(int)
for word_id, word_count in itertools.chain.from_iterable(corpus):
    total_word_count[word_id] += word_count

# Create a sorted list from the defaultdict: sorted_word_count
sorted_word_count = sorted(total_word_count.items(), key=lambda w: w[1], reverse=True) 

# Print the top 5 words across all documents alongside the count
for word_id, word_count in sorted_word_count[:5]:
    print(dictionary.get(word_id), word_count)


movie 51693
film 47031
one 27740
like 20740
time 15959


Por se tratar de um conjunto de _review_ de filmes é comum que tenhamos algumas palavras que sejam _muito_ frequentes em todos os documentos. Palavras como _movie_, _film_ ou _character_ deve ser bem comuns, expremindo pouco significado sobre o _review_ que vamos analisar. Por exemplo, fica claro que o _review 0_ é um filme sobre o Michael Jackson

In [146]:
 train["review"][0] 

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

Enquanto o _review 1_ diz respeito a um filme de ficção que envolve guerra e é baseado em um livro

In [151]:
 train["review"][1] 

'"\\"The Classic War of the Worlds\\" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells\' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \\"critics\\" look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \\"critics\\". We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells\' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \\"critics\\" perceive to be its shortcomings."'

Percebemos isso por meio de palavras mais marcantes, como _MJ_ no caso do primeiro review e _war world_ ou _book_ no caso do segundo. Não seria mais interessante se pudessemos dar um peso maior para a frequência das palavras mais "raras", que trazem maior signifcado para o texto?

E podemos fazer isso ! Essa métrica é conhecida como $tf-idf_t$.

### O tf-idf

Essa métrica é conhecida pode ser calculada por: $tf-idf_{t,d} = tf_{t,d} × idf_{t}$.

Mas o que ela é? 

-$t_f$ é o _term frequency_, ou seja, a frequência do termo naquele documento, conforme vimos.

-$idf_{t}$ é o inverso da frequência do termo considerando todos os documentos. Em outras palavras, considerando o número total de documentos na nossa coleção de documentos, $N$ e o número de documentos que contém o termo $t$, como $d_t$, ela é dada por $log(\frac{N}{df_i})$ . Isso ocorre de forma que palavras **muito** frequentes tenham um idf **baixo** e palavras **pouco** frequentes tenham um idf **alto**.

Para ilustrar, suponha que uma palavra apareça em **todos** os documentos, o resultado de $\frac{N}{df_i}$ será 1 e o $log(1) = 0$. Ao contrário, se uma palavra aparece em um **único** documento, o resultado de $\frac{N}{df_i}$ será perto de $N$, sendo $N$ a medida de _peso_ a ser aplicada à frequência $tf_{t,d}$.

Logo, a nossa métrica $tf-idf_{t,d}$ pode ser calculada como:

$tf-idf_{t,d} = tf_{t,d} × log(\frac{N}{df_i})$

Mas você precisa fazer isso **toda** vez que quiser calcular essa medida? Não !

Veremos como você pode fazer isso usando o _gensim_ e no _sklearn_.

In [89]:
from gensim.models.tfidfmodel import TfidfModel

In [90]:
tfidf = TfidfModel(corpus)

In [91]:
doc = corpus[0]
tfidf_weights = tfidf[doc]
sorted_tfidf_weights = sorted(tfidf_weights, key=lambda w: w[1], reverse=True)

# Print the top 5 weighted words
for term_id, weight in sorted_tfidf_weights[:5]:
    print(dictionary.get(term_id), weight)

mj 0.8072471966961522
moonwalker 0.13795536312501597
pesci 0.13376938648072365
drug 0.1082807265701741
message 0.09803922131947461


_MJ_ tem o tf-idf mais alto, corroborando o que falamos anteriormente.

In [200]:
from sklearn.feature_extraction.text import TfidfVectorizer 
# Initialize a TfidfVectorizer object: tfidf_vectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer = "word", \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)  



tfidf_train_vocab = tfidf_vectorizer.fit(clean_train_reviews)
tfidf_train_vocab.vocabulary_

{'stuff': 4295,
 'going': 1960,
 'moment': 2907,
 've': 4750,
 'started': 4221,
 'listening': 2640,
 'music': 2959,
 'watching': 4846,
 'odd': 3097,
 'documentary': 1345,
 'watched': 4844,
 'maybe': 2798,
 'want': 4823,
 'get': 1930,
 'certain': 751,
 'insight': 2322,
 'guy': 2037,
 'thought': 4496,
 'really': 3586,
 'cool': 1028,
 'eighty': 1467,
 'make': 2734,
 'mind': 2868,
 'whether': 4885,
 'guilty': 2032,
 'innocent': 2318,
 'part': 3205,
 'biography': 503,
 'feature': 1709,
 'film': 1745,
 'remember': 3655,
 'see': 3904,
 'cinema': 838,
 'originally': 3139,
 'released': 3640,
 'subtle': 4313,
 'message': 2838,
 'feeling': 1715,
 'towards': 4569,
 'press': 3407,
 'also': 216,
 'obvious': 3088,
 'drug': 1400,
 'bad': 400,
 'kay': 2471,
 'visually': 4791,
 'impressive': 2271,
 'course': 1056,
 'michael': 2847,
 'jackson': 2397,
 'unless': 4700,
 'remotely': 3662,
 'like': 2620,
 'anyway': 278,
 'hate': 2086,
 'find': 1753,
 'boring': 559,
 'may': 2797,
 'call': 668,
 'making': 2737

In [201]:
len(tfidf_train_vocab.vocabulary_)

5000

In [202]:
train_data_features = tfidf_vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

In [203]:
train_data_features[0][4292:4296] # Vai do indice 4292 ate o 4295

array([ 0.        ,  0.        ,  0.        ,  0.06198476])

In [204]:
train_data_features.shape

(25000, 5000)

## Realizando a analise de sentimentos !


Agora, nós temos as features geradas pelo _tfidf_ aplicado ao nosso modelo de _Bag of Words_ e o sentimento original para cada vetor de _features_. Por termos as _labels_, podemos realizar aprendizado surpervisionado !

Vocês aprenderam _Random Forest_ em uma das aulas passadas, então vamos usar ele aqui. Usaremos 100 árvores, o valor _default_ do _scikit_. Usamos 5000  features (top 5000 palavras) por uma questão de otimização. Poderíamos ter usado mais, mas levaria um tempo consideravelmente maior.

Usamos Random Forest, mas poderíamos ter usado qualquer outro método de aprendizado surpervisionado. Em texto é comum trabalharmos com uma abordagem probabilística conhecida como _Naïve Bayes_.



## Testando o que desenvolvemos

Note que quando nós usamos o _Bag of Words_ para o caso de teste, nós apenas chamamos a função _transform_ uma vez que não queremos aprender o vocabulário do caso de teste, ao contrário do que fizemos no caso de treino, em que chamamos _fit_transform_. Relembrando, nós fazemos isso para evitar _overfitting_.





  0%|          | 23/25000 [00:00<01:49, 227.13it/s]

(25000, 2)


100%|██████████| 25000/25000 [01:15<00:00, 329.48it/s]


id                                             review  \
0      "12311_10"  "Naturally in a film who's main themes are of ...   
1        "8348_2"  "This movie is a disaster within a disaster fi...   
2        "5828_4"  "All in all, this is a movie for kids. We saw ...   
3        "7186_2"  "Afraid of the Dark left me with the impressio...   
4       "12128_7"  "A very accurate depiction of small time mob l...   
5        "2913_8"  "...as valuable as King Tut's tomb! (OK, maybe...   
6        "4396_1"  "This has to be one of the biggest misfires ev...   
7         "395_2"  "This is one of those movies I watched, and wo...   
8       "10616_1"  "The worst movie i've seen in years (and i've ...   
9        "9074_9"  "Five medical students (Kevin Bacon, David Lab...   
10       "9252_3"  "'The Mill on the Floss' was one of the lesser...   
11       "9896_9"  "I just saw this film at the phoenix film fest...   
12        "574_4"  "\"The Love Letter\" is one of those movies th...   
13      "11182_8"  "Another fantastic offering from the Monkey Is...   
14      "11656_4"  "This was included on the disk \"Shorts: Volum...   
15       "2322_4"  "I'm not really much of an Abbott & Costello f...   
16       "8703_1"  "This movie was dreadful. Biblically very inac...   
17       "7483_1"  "I don't think I've ever gave something a 1/10...   
18      "6007_10"  "Excellent story-telling and cinematography. P...   
19      "12424_4"  "I completely forgot that I'd seen this within...   
20       "4672_1"  "I like action movies. I have a softspot for \...   
21      "10841_3"  "This is one of the worst Sandra Bullock movie...   
22       "8954_7"  "Watched this flick on Saturday afternoon cabl...   
23       "7392_1"  "I went to see \"TKIA\" with high expectations...   
24      "10288_8"  "All credit to writer/director Gilles Mimouni ...   
25       "5343_4"  "As a writing teacher, there are two ending I ...   
26       "4950_1"  "I don't know why this has gotten any decent r...   
27       "9257_4"  "This film was released in the UK under the na...   
28       "8689_3"  "Uncle Fred Olen Ray once again gives us a lit...   
29       "4480_2"  "OK, it's watchable if you are sick in bed or ...   
...           ...                                                ...   
24970   "6857_10"  "With \"Anatomy\" the german film producers ha...   
24971   "11091_8"  "This movie is one of my all-time favorites. I...   
24972    "4167_2"  "I found Code 46 very disappointing. I thought...   
24973     "679_4"  "Tamara Anderson and her family are moving onc...   
24974   "10147_1"  "Now I've seen it all. Just when I thought it ...   
24975    "6875_1"  "In this movie everything possible was wrong a...   
24976    "923_10"  "Well every scene so perfectly presented. Neve...   
24977    "6200_8"  "Sleeper Cell is what 24 should have been. 24 ...   
24978    "7208_8"  "Not for everyone, but I really like it. Nice ...   
24979    "5363_8"  "Set just before the Second World War, this is...   
24980    "4067_8"  "Contains Spoiler The movie is a good action/c...   
24981    "1773_7"  "This is one of several period sea-faring yarn...   
24982   "1498_10"  "Hearkening back to those \"Good Old Days\" of...   
24983  "10497_10"  "I thought this to be a pretty good example of...   
24984   "3444_10"  "Seeing this film, or rather set of films, in ...   
24985     "588_2"  "I didn't like this movie for many reasons - V...   
24986    "9678_9"  "I absolutely love this show!!!!!!!, Its basic...   
24987    "1983_9"  "eXistenZ combines director David Cronenberg's...   
24988    "5012_3"  "this movie is allegedly a comedy.so where did...   
24989   "12240_2"  "The Comebacks is a spoof on inspirational spo...   
24990    "5071_2"  "I'd love to write a little summary of this mo...   
24991    "5078_2"  "Obvious tailored vehicle for Ryan Philippe. I...   
24992   "10069_3"  "<br /><br />JURASSIC PARK III *___ Adventure ...   
24993    "7407_8"  "If you're even mildly interested in